In [ ]:
#Written_By_Eng. Mohammed Albhaisi M.Sc.

from nuscenes import NuScenes

#Load ground truth data
nusc = NuScenes(version='v1.0-trainval', dataroot='/home/albhaisi/Desktop/WS/data', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 29.119 seconds.
Reverse indexing ...
Done reverse indexing in 6.2 seconds.


In [3]:
#Load_results
import json
from nuscenes.eval.common.data_classes import EvalBoxes
from nuscenes.eval.tracking.data_classes import TrackingBox

#give tracker path
result_path = '/home/albhaisi/Desktop/Presentation/all_tracking_results/pointpillar/4bb9626f13184da7a67ba1b241c19558/tracking_results_car_scene_4bb9626f13184da7a67ba1b241c19558.json'

def load_predictions(result_path):
        #Choose which scene results to upload 
        with open(result_path) as f:
                data = json.load(f)
                result = data['results']
        all_results = EvalBoxes.deserialize(data['results'], TrackingBox)
        print(all_results)
        meta = data['meta'] 
        # How much sample in this result and number of boxes
        print("Loaded results from {}. Found detections for {} samples."
              .format(result_path, len(all_results.sample_tokens)))

        return result

prediction_results= load_predictions(result_path)

#Check_results
print(prediction_results)

EvalBoxes with 1660 boxes across 40 samples
Loaded results from /home/albhaisi/Desktop/Presentation/all_tracking_results/pointpillar/4bb9626f13184da7a67ba1b241c19558/tracking_results_car_scene_4bb9626f13184da7a67ba1b241c19558.json. Found detections for 40 samples.
{'bb10039077c74099b51fb67300391659': [{'translation': [1300.2469919549044, 888.1753050523215, 0.9516881945193268], 'size': [1.9295341968536377, 4.469816207885742, 1.636683702468872], 'velocity': [0.0, 0.0], 'rotation': [0.817602969759909, 0.0, 0.0, 0.5757824101514195], 'tracking_score': 0.8381890058517456, 'sample_token': 'bb10039077c74099b51fb67300391659', 'tracking_id': 'car_63_0', 'tracking_name': 'car'}, {'translation': [1302.5481681500269, 886.319622778136, 0.9528341413647629], 'size': [1.933990240097046, 4.485490322113037, 1.6273444890975952], 'velocity': [0.0, 0.0], 'rotation': [0.826109577384181, 0.0, 0.0, 0.563509508485997], 'tracking_score': 0.7835906744003296, 'sample_token': 'bb10039077c74099b51fb67300391659', 'tr

In [4]:
from collections import defaultdict
from typing import List, Dict, Tuple

import numpy as np

from nuscenes.eval.common.data_classes import MetricData, EvalBox
from nuscenes.eval.common.utils import center_distance, scale_iou
from nuscenes.eval.detection.constants import DETECTION_NAMES, ATTRIBUTE_NAMES, TP_METRICS

#Edit Detectionbox

class DetectionBox(EvalBox):
    """ Data class used during detection evaluation. Can be a prediction or ground truth."""

    def __init__(self,
                 sample_token: str = "",                 
                 sample_annotation_token: str = "", # sample_annotation_token only for gt boxes.
                 translation: Tuple[float, float, float] = (0, 0, 0),
                 size: Tuple[float, float, float] = (0, 0, 0),
                 rotation: Tuple[float, float, float, float] = (0, 0, 0, 0),
                 velocity: Tuple[float, float] = (0, 0),
                 ego_translation: [float, float, float] = (0, 0, 0),  # Translation to ego vehicle in meters.
                 num_pts: int = -1,  # Nbr. LIDAR or RADAR inside the box. Only for gt boxes.
                 tracking_name: str = 'car',  # The class name used in the detection challenge.
                 detection_name: str = 'car',
                 detection_score: float = -1.0,
                 tracking_score: float = -1.0):  # GT samples do not have a score.
                 
          

        super().__init__(sample_token, translation, size, rotation, velocity, ego_translation, num_pts)

        assert tracking_name is not None, 'Error: detection_name cannot be empty!'
        assert tracking_name in DETECTION_NAMES, 'Error: Unknown detection_name %s' % tracking_name


        assert type(tracking_score) == float, 'Error: detection_score must be a float!'
        assert not np.any(np.isnan(tracking_score)), 'Error: detection_score may not be NaN!'

        # Assign.
        self.tracking_name = tracking_name
        self.detection_name= detection_name
        self.tracking_score = tracking_score
        self.detection_score = detection_score
    def __eq__(self, other):
        return (self.sample_token == other.sample_token and
                self.sample_annotation_token == other.sample_annotation_token and
                self.translation == other.translation and
                self.size == other.size and
                self.rotation == other.rotation and
                self.velocity == other.velocity and
                self.ego_translation == other.ego_translation and
                self.num_pts == other.num_pts and
                self.tracking_name == other.tracking_name and
                self.detection_name == other.detection_name and
                self.detection_score == other.detection_score and
                self.tracking_score == other.tracking_score)

    def serialize(self) -> dict:
        """ Serialize instance into json-friendly format. """
        return {
            'sample_token': self.sample_token,
            'sample_annotation_token': self.sample_annotation_token,
            'translation': self.translation,
            'size': self.size,
            'rotation': self.rotation,
            'velocity': self.velocity,
            'ego_translation': self.ego_translation,
            'ego_dist': self.ego_dist, # return ego distance as well
            'num_pts': self.num_pts,
            'tracking_name': self.tracking_name,
            'detection_name': self.detection_name,
            'tracking_score': self.tracking_score,
            'detection_score': self.detection_score
    
        }

    @classmethod
    def deserialize(cls, content: dict):
        """ Initialize from serialized content. """
        return cls(sample_token=content['sample_token'],
                   sample_annotation_token= "" if 'sample_annotation_token' not in content
                   else content['sample_annotation_token'],
                   translation=tuple(content['translation']),
                   size=tuple(content['size']),
                   rotation=tuple(content['rotation']),
                   velocity=tuple(content['velocity']),
                   ego_translation=(0.0, 0.0, 0.0) if 'ego_translation' not in content
                   else tuple(content['ego_translation']),
                   num_pts=-1 if 'num_pts' not in content else int(content['num_pts']),
                   tracking_name=content['tracking_name'],
                   detection_name=content['tracking_name'],
                   detection_score=-1.0 if 'detection_score' not in content else float(content['tracking_score']),
                   tracking_score=-1.0 if 'tracking_score' not in content else float(content['tracking_score']))

In [5]:
#Load_ground_truth_for_one_scene

from typing import Dict, Tuple
import numpy as np
import tqdm
from pyquaternion import Quaternion
from nuscenes.eval.detection.utils import category_to_detection_name
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import points_in_box
from nuscenes.utils.splits import create_splits_scenes
from nuscenes.eval.common.data_classes import EvalBoxes
from nuscenes.eval.tracking.data_classes import TrackingBox


eval_set: str
verbose: bool = True

eval_set='val'

def get_all_samples_in_scene():
        all_samples_in_scene =[]
        #To know number of annotation for all instances
        for samples in prediction_results:
                all_samples_in_scene.append(samples)
        return all_samples_in_scene


sample_boxes = []

def load_gt(nusc: NuScenes, eval_split: str, box_cls, verbose: bool = False) -> EvalBoxes:
    """
    Loads ground truth boxes from DB.
    :param nusc: A NuScenes instance.
    :param eval_split: The evaluation split for which we load GT boxes.
    :param box_cls: Type of box to load, e.g. DetectionBox or TrackingBox.
    :param verbose: Whether to print messages to stdout.
    :return: The GT boxes.
    """
    # Init.
    if box_cls == DetectionBox:
        attribute_map = {a['token']: a['name'] for a in nusc.attribute}

    if verbose:
        print('Loading annotations for {} split from nuScenes version: {}'.format(eval_split, nusc.version))
    # Read out all sample_tokens in DB.
    sample_tokens_all = get_all_samples_in_scene()
    assert len(sample_tokens_all) > 0, "Error: Database has no samples!"

    # Only keep samples from this split.
    splits = create_splits_scenes()

    # Check compatibility of split with nusc_version.
    version = nusc.version
    if eval_split in {'train', 'val', 'train_detect', 'train_track'}:
        assert version.endswith('trainval'), \
            'Error: Requested split {} which is not compatible with NuScenes version {}'.format(eval_split, version)
    elif eval_split in {'mini_train', 'mini_val'}:
        assert version.endswith('mini'), \
            'Error: Requested split {} which is not compatible with NuScenes version {}'.format(eval_split, version)
    elif eval_split == 'test':
        assert version.endswith('test'), \
            'Error: Requested split {} which is not compatible with NuScenes version {}'.format(eval_split, version)
    else:
        raise ValueError('Error: Requested split {} which this function cannot map to the correct NuScenes version.'
                         .format(eval_split))

    if eval_split == 'test':
        # Check that you aren't trying to cheat :).
        assert len(nusc.sample_annotation) > 0, \
            'Error: You are trying to evaluate on the test set but you do not have the annotations!'

    sample_tokens = []
    for sample_token in sample_tokens_all:
        scene_token = nusc.get('sample', sample_token)['scene_token']
        scene_record = nusc.get('scene', scene_token)
        if scene_record['name'] in splits[eval_split]:
            sample_tokens.append(sample_token)

    all_annotations = EvalBoxes()

    # Load annotations and filter predictions and annotations.
    tracking_id_set = set()
    for sample_token in tqdm.tqdm(sample_tokens, leave=verbose):

        sample = nusc.get('sample', sample_token)
        sample_annotation_tokens = sample['anns']

        
        for sample_annotation_token in sample_annotation_tokens:

            sample_annotation = nusc.get('sample_annotation', sample_annotation_token)
            if box_cls == DetectionBox:
                # Get label name in detection task and filter unused labels.
                tracking_name = category_to_detection_name(sample_annotation['category_name'])
                if tracking_name is None:
                    continue


                if tracking_name == 'car':
                    sample_boxes.append(
                        box_cls(
                            sample_token=sample_token,
                            translation=sample_annotation['translation'],
                            size=sample_annotation['size'],
                            rotation=sample_annotation['rotation'],
                            velocity=nusc.box_velocity(sample_annotation['token'])[:2],
                            num_pts=sample_annotation['num_lidar_pts'] + sample_annotation['num_radar_pts'],
                            tracking_name=tracking_name,
                            tracking_score=-1.0,  # GT samples do not have a score.
                        )
                    )
            elif box_cls == TrackingBox:
                # Use nuScenes token as tracking id.
                tracking_id = sample_annotation['instance_token']
                tracking_id_set.add(tracking_id)

                # Get label name in detection task and filter unused labels.
                # Import locally to avoid errors when motmetrics package is not installed.
                from nuscenes.eval.tracking.utils import category_to_tracking_name
                tracking_name = category_to_tracking_name(sample_annotation['category_name'])
                if tracking_name is None:
                    continue
                if tracking_name == 'car':
                    sample_boxes.append(
                        box_cls(
                            sample_token=sample_token,
                            translation=sample_annotation['translation'],
                            size=sample_annotation['size'],
                            rotation=sample_annotation['rotation'],
                            velocity=nusc.box_velocity(sample_annotation['token'])[:2],
                            num_pts=sample_annotation['num_lidar_pts'] + sample_annotation['num_radar_pts'],
                            tracking_id=tracking_id,
                            tracking_name=tracking_name,
                            tracking_score=-1.0  # GT samples do not have a score.
                        )
                    )
                
            else:
                raise NotImplementedError('Error: Invalid box_cls %s!' % box_cls)

        all_annotations.add_boxes(sample_token, sample_boxes)

    if verbose:
        print("Loaded ground truth annotations for {} samples.".format(len(all_annotations.sample_tokens)))

    return all_annotations

gt_boxes = load_gt(nusc, eval_set, TrackingBox, verbose=verbose)



Loading annotations for val split from nuScenes version: v1.0-trainval


100%|██████████| 40/40 [00:00<00:00, 113.59it/s]

Loaded ground truth annotations for 40 samples.


In [6]:
for boxess in sample_boxes:
    print(boxess)



{'sample_token': 'bb10039077c74099b51fb67300391659', 'translation': [1302.675, 886.457, 0.861], 'size': [1.618, 4.35, 1.511], 'rotation': [0.8137707789099915, 0.0, 0.0, 0.58118595939013], 'velocity': array([-0.00200049,  0.        ]), 'ego_translation': (0, 0, 0), 'num_pts': 67, 'tracking_id': '786a260f8a67425dbe83b2d2feaa880c', 'tracking_name': 'car', 'tracking_score': -1.0}
{'sample_token': 'bb10039077c74099b51fb67300391659', 'translation': [1331.802, 878.115, 0.786], 'size': [1.921, 4.96, 2.0], 'rotation': [0.35222554575392123, 0.0, 0.0, 0.9359151483539265], 'velocity': array([-10.74864468,   8.50609291]), 'ego_translation': (0, 0, 0), 'num_pts': 0, 'tracking_id': 'e28bb1776e354f51b520d33c01a5e722', 'tracking_name': 'car', 'tracking_score': -1.0}
{'sample_token': 'bb10039077c74099b51fb67300391659', 'translation': [1297.112, 899.586, 0.984], 'size': [1.966, 4.405, 2.0], 'rotation': [0.9405877693367394, 0.0, 0.0, -0.33955065627110315], 'velocity': array([0., 0.]), 'ego_translation': (

: 

In [11]:
#Load_results
import json

from nuscenes.eval.tracking.data_classes import TrackingConfig
from nuscenes.eval.detection.data_classes import DetectionConfig

#Upload tracking config
with open('/home/albhaisi/Desktop/Presentation/for_automatic_test/results/tracking_nips_2023.json', 'r') as _f:
    cfg = TrackingConfig.deserialize(json.load(_f))

#Upload detection config
with open('/home/albhaisi/Desktop/Presentation/for_automatic_test/base_codes/Final_codes/detection_cvpr_2023.json', 'r') as _f:
    cfg2 = DetectionConfig.deserialize(json.load(_f))

def load_prediction(result_path: str, max_boxes_per_sample: int, box_cls, verbose: bool = False) \
        -> Tuple[EvalBoxes, Dict]:
    """
    Loads object predictions from file.
    :param result_path: Path to the .json result file provided by the user.
    :param max_boxes_per_sample: Maximim number of boxes allowed per sample.
    :param box_cls: Type of box to load, e.g. DetectionBox or TrackingBox.
    :param verbose: Whether to print messages to stdout.
    :return: The deserialized results and meta data.
    """

    # Load from file and check that the format is correct.
    with open(result_path) as f:
        data = json.load(f)
    assert 'results' in data, 'Error: No field `results` in result file. Please note that the result format changed.' \
                              'See https://www.nuscenes.org/object-detection for more information.'

    # Deserialize results and get meta data.
    all_results = EvalBoxes.deserialize(data['results'], box_cls)
    meta = data['meta']
    if verbose:
        print("Loaded results from {}. Found detections for {} samples."
              .format(result_path, len(all_results.sample_tokens)))

    # Check that each sample has no more than x predicted boxes.
    for sample_token in all_results.sample_tokens:
        assert len(all_results.boxes[sample_token]) <= max_boxes_per_sample, \
            "Error: Only <= %d boxes per sample allowed!" % max_boxes_per_sample

    return all_results, meta


pred_boxes, meta = load_prediction(result_path, cfg.max_boxes_per_sample, TrackingBox,
                                                verbose=verbose)
print(load_prediction(result_path,cfg.max_boxes_per_sample,TrackingBox))

pred_boxes2, meta2 = load_prediction(result_path, cfg.max_boxes_per_sample, DetectionBox,
                                                     verbose=verbose)
gt_boxes2 = load_gt(nusc, eval_set, DetectionBox, verbose=verbose)

assert set(pred_boxes2.sample_tokens) == set(gt_boxes2.sample_tokens), \
            "Samples in split don't match samples in predicted tracks."


Loaded results from /home/albhaisi/Desktop/Presentation/all_tracking_results/centerpoint_01voxel/d1e57234fd6a463d963670938f9f556e/tracking_results_car_scene_d1e57234fd6a463d963670938f9f556e.json. Found detections for 40 samples.
(EvalBoxes with 223 boxes across 40 samples, {'use_camera': False, 'use_lidar': True, 'use_radar': False, 'use_map': False, 'use_external': False})
Loaded results from /home/albhaisi/Desktop/Presentation/all_tracking_results/centerpoint_01voxel/d1e57234fd6a463d963670938f9f556e/tracking_results_car_scene_d1e57234fd6a463d963670938f9f556e.json. Found detections for 40 samples.
Loading annotations for val split from nuScenes version: v1.0-trainval


100%|██████████| 40/40 [00:00<00:00, 47893.85it/s]

Loaded ground truth annotations for 40 samples.


In [12]:
from nuscenes.eval.common.loaders import add_center_dist,filter_eval_boxes
from nuscenes.eval.tracking.loaders import create_tracks 
from nuscenes.eval.detection.data_classes import DetectionMetricDataList
from nuscenes.eval.detection.algo import accumulate, calc_ap, calc_tp
from nuscenes.eval.detection.constants import TP_METRICS
from nuscenes.eval.detection.render import summary_plot, class_pr_curve, class_tp_curve, dist_pr_curve, visualize_sample

   # Add center distances.
pred_boxes = add_center_dist(nusc, pred_boxes)
gt_boxes = add_center_dist(nusc, gt_boxes)

# -----------------------------------
# Step 1: Accumulate metric data for all classes and distance thresholds.
# -----------------------------------
if verbose:
    print('Accumulating metric data...')
metric_data_list = DetectionMetricDataList()

for class_name in cfg2.class_names:
    for dist_th in cfg2.dist_ths:
        continue
     


        
    
    


Accumulating metric data...


In [13]:
#Initialization

from nuscenes.eval.common.loaders import add_center_dist,filter_eval_boxes
from nuscenes.eval.tracking.loaders import create_tracks

# Add center distances.
pred_boxes = add_center_dist(nusc, pred_boxes)
gt_boxes = add_center_dist(nusc, gt_boxes)

print(pred_boxes)
print(gt_boxes)

# Filter boxes (distance, points per box, etc.).
if verbose:
            print('Filtering tracks')
            pred_boxes = filter_eval_boxes(nusc, pred_boxes, cfg.class_range, verbose=verbose)
if verbose:
            print('Filtering ground truth tracks')
            gt_boxes = filter_eval_boxes(nusc, gt_boxes, cfg.class_range, verbose=verbose)

            sample_tokens = gt_boxes.sample_tokens

            # Convert boxes to tracks format.
            tracks_gt = create_tracks(gt_boxes, nusc, eval_set, gt=True)
            tracks_pred = create_tracks(pred_boxes, nusc, eval_set, gt=False)



EvalBoxes with 223 boxes across 40 samples
EvalBoxes with 42 boxes across 40 samples
Filtering tracks
=> Original number of boxes: 223
=> After distance based filtering: 218
=> After LIDAR and RADAR points based filtering: 218
=> After bike rack filtering: 218
Filtering ground truth tracks
=> Original number of boxes: 42
=> After distance based filtering: 42
=> After LIDAR and RADAR points based filtering: 42
=> After bike rack filtering: 42


In [18]:
#Detection
class_name='car'
npos = len([1 for gt_boxes2 in gt_boxes.all if gt_boxes2.tracking_name == 'car'])
if verbose:
        print("Found {} GT of class {} out of {} total across {} samples.".
              format(npos, class_name, len(gt_boxes.all), len(gt_boxes.sample_tokens)))
# Organize the predictions in a single list.
pred_boxes_list = [box for box in pred_boxes2.all if box.tracking_name == 'car']
pred_confs = [box.tracking_score for box in pred_boxes_list]
print(pred_confs)
if verbose:
        print("Found {} PRED of class {} out of {} total across {} samples.".
              format(len(pred_confs), class_name, len(pred_boxes2.all), len(pred_boxes2.sample_tokens)))
                            
# Sort by confidence.
sortind = [i for (v, i) in sorted((v, i) for (i, v) in enumerate(pred_confs))][::-1]
#print(sortind)
#Sort by annotations
sortind2 = [i for (v, i) in ((v, i) for (i, v) in enumerate(pred_confs))]

# Do the actual matching.
tp = []  # Accumulator of true positives.
fp = []  # Accumulator of false positives.
conf = []  # Accumulator of confidences.


# match_data holds the extra metrics we calculate for each match.
match_data = {'trans_err': [],
                  'vel_err': [],
                  'scale_err': [],
                  'orient_err': [],
                  'attr_err': [],
                  'IOU': [],
                  'conf': []}


Found 42 GT of class car out of 42 total across 40 samples.
[0.21013492345809937, 0.17175383865833282, 0.14774273335933685, 0.14646948873996735, 0.12456749379634857, 0.10784395039081573, 0.14755509793758392, 0.0017175383865833283, 0.0014774273335933685, 0.12286602705717087, 0.12153809517621994, 0.2593854069709778, 0.30045434832572937, 0.10369835793972015, 0.18117693066596985, 0.19602316617965698, 0.11306720972061157, 0.3135117292404175, 0.40245500206947327, 0.0010369835793972017, 0.1672646701335907, 0.18726637959480286, 0.10142592340707779, 0.26284515857696533, 0.30628684163093567, 0.13111431896686554, 0.15398716926574707, 0.15272554755210876, 0.1994742453098297, 0.18788549304008484, 0.2867545783519745, 0.0013111431896686554, 0.13994327187538147, 0.12377531826496124, 0.22927770018577576, 0.23199200630187988, 0.10973120480775833, 0.295643150806427, 0.26808738708496094, 0.11631380021572113, 0.0013994327187538146, 0.14020241796970367, 0.18973004817962646, 0.14561299979686737, 0.1449386775

In [19]:
from typing import Callable
from nuscenes.eval.common.utils import center_distance, scale_iou, yaw_diff, velocity_l2, attr_acc, cummean
from nuscenes.eval.detection.data_classes import DetectionMetricDataList


    




# ---------------------------------------------
# Match and accumulate match data.
# ---------------------------------------------

taken = set()  # Initially no gt bounding box is matched.
for ind in sortind2:
    pred_box = pred_boxes_list[ind]
    min_dist = np.inf
    match_gt_idx = None
    for gt_idx, gt_box in enumerate(gt_boxes[pred_box.sample_token]):
        # Find closest match among ground truth boxes
        if gt_box.tracking_name == class_name and not (pred_box.sample_token, gt_idx) in taken:
            this_distance = cfg.dist_fcn_callable(gt_box, pred_box)
            if this_distance < min_dist:
                min_dist = this_distance
                match_gt_idx = gt_idx
                # add reference to closest annotation
                #sample_annotation_token = gt_boxes2.sample_annotation_token

    # If the closest match is close enough according to threshold we have a match!
    is_match = min_dist < dist_th

    if is_match:
        taken.add((pred_box.sample_token, match_gt_idx))
        #  Update tp, fp and confs.
        tp.append(1)
        fp.append(0)
        conf.append(pred_box.tracking_score)
        # Since it is a match, update match data also.
        gt_box_match = gt_boxes[pred_box.sample_token][match_gt_idx]
        match_data['IOU'].append(scale_iou(gt_box_match, pred_box))
        match_data['scale_err'].append(1 - scale_iou(gt_box_match, pred_box))
        match_data['trans_err'].append(center_distance(gt_box_match, pred_box))
        period = np.pi if class_name == 'barrier' else 2 * np.pi
        match_data['orient_err'].append(yaw_diff(gt_box_match, pred_box, period=period))
        match_data['conf'].append(pred_box.tracking_score)
        #print(yaw_diff(gt_box_match, pred_box, period=period))
    
#test
print(match_data['IOU'])
print(match_data['orient_err'])
print(match_data['trans_err'])
print(match_data['conf'])
#print(match_data['vis'])
        

if verbose:
            print('Accumulating metric data...')
metric_data_list = DetectionMetricDataList()


             


[0.750937407253894, 0.750937407253894, 0.8796264863640975, 0.8546812935499198, 0.8819319601824613, 0.8216312388479655, 0.8242342444191073, 0.8216312388479655, 0.6711718314103643, 0.8686181268116604, 0.6711718314103643, 0.7569379165394748, 0.8567431530575554]
[3.1094304683225955, 3.1094304683225955, 0.14590345290084583, 0.14083995612823452, 0.09063982445811902, 0.09250002942388491, 2.960000982467191, 0.11867996820380045, 3.1288715852306535, 0.017092353006255223, 3.1288715852306535, 0.07938420358232978, 0.1860214885580591]
[2.7114829686884407, 2.7114829686884407, 1.3084200128659758, 0.485738061505748, 0.09239483634545079, 0.6344807554318497, 1.5401768703385919, 2.9289336037541833, 3.899505675095245, 0.8738726849314291, 3.899505675095245, 2.066393187523066, 2.6330825644354467]
[0.18745917081832886, 0.0018745917081832886, 0.4451814889907837, 0.16105134785175323, 0.5664065480232239, 0.502911388874054, 0.5461122393608093, 0.0050291138887405395, 0.10781558603048325, 0.20214131474494934, 0.001

: 

In [25]:

# ---------------------------------------------
# Calculate and interpolate precision and recall
# ---------------------------------------------
# Do the actual matching.
tp = []  # Accumulator of true positives.
fp = []  # Accumulator of false positives.
conf = []  # Accumulator of confidences.
# Accumulate.
tp = np.cumsum(tp).astype(float)
fp = np.cumsum(fp).astype(float)
conf = np.array(conf)

# Calculate precision and recall.
prec = tp / (fp + tp)
rec = tp / float(npos)




In [26]:
from nuscenes.eval.detection.data_classes import DetectionConfig, DetectionMetrics
import os

_label_aps = defaultdict(lambda: defaultdict(float))
metrics = DetectionMetrics(cfg2)

#Where to save results
output_dir = '/home/albhaisi/Desktop/Presentation/for_automatic_test/base_codes/test'

def mean_dist_aps():
        """ Calculates the mean over distance thresholds for each label. """
        return {class_name: np.mean(list(d.values())) for class_name, d in _label_aps.items()}

def mean_ap(self) -> float:
        """ Calculates the mean AP by averaging over distance thresholds and classes. """
        return float(np.mean(list(mean_dist_aps.values())))


if verbose:
    print('Saving metrics to: %s' % output_dir)

metrics_summary = metrics.serialize()
metrics_summary['meta'] = meta.copy()

with open(os.path.join(output_dir, 'metrics_details.json'), 'w') as f:
            f.write('Final_metrics_calculation: \n \n')

            f.write('Confidence_scor: \n \n')
            json.dump(match_data['conf'], f, indent=2)

            f.write('IOU: \n \n')
            json.dump(match_data['IOU'], f, indent=2)

            f.write('Translation_error: \n \n')
            json.dump(match_data['trans_err'], f, indent=2)
            
            f.write('Orientation_error: \n \n')
            json.dump(match_data['orient_err'], f, indent=2)



Saving metrics to: /home/albhaisi/Desktop/Presentation/for_automatic_test/base_codes/test


/home/albhaisi/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/albhaisi/.local/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
